In [200]:
import code
import numpy as np
import ntpath
from pathlib import Path
import os
from baseline_model import * 
#from particle import Particle_Filter 
import pandas as pd
import math
#from filterpy.monte_carlo import systematic_resample
import torch

In [201]:
root_path = Path('./../../..')

config = {}
config['model'] = root_path / 'baboon_tracking' / 'models' / 'particle_filter' / 'net.pth'
config['input_dim'] = 9
config['output_dim'] = -1 
config['kmeans_model_path'] = root_path / 'ml_data' / 'velocity_model.pkl'
config['input_csv'] = root_path / 'ml_data' / 'input_mp4.csv'
config['video_csv'] = root_path / 'ml_data' / '4_22_2020_mask.mp4_blobdetector.csv'

########## csv Initialization ##########
data = pd.read_csv(config['input_csv'])
video = pd.read_csv(config['video_csv'])
del video['Unnamed: 0']

In [202]:
class Particle_Filter():
    def __init__(self, estimated_location, init_velocities, weights, estimated_velocity, direction_vector, des_num_particles):
        # velocities
        self.velocities = init_velocities
        self.weights = weights
        
        # estimated location of baboon at initialization
        self.estimated_location = estimated_location
        self.estimated_velocity = estimated_velocity
        self.direction_vector = direction_vector
        self.des_num_particles = des_num_particles
                
    #frame - n x 2 matrix of all blob detected centroids in the current frame
    #threshold - pixel distance from the previous position that points should be considered from the blob detector
    #alpha - degree of trust in blob detector (probably 0.8-0.9)
    def update(self, frame, threshold, alpha=0.8, noise=8, FPS=30):
        predict_velocities = self.velocities
        predict_weights = self.weights
        #print("Starting velocity")
        #print(predict_velocities)
        #print("Starting weights")
        #print(predict_weights)
        prev_position = self.estimated_location
        prev_velocity = self.estimated_velocity
        new_velocities = []
        new_weights = []
        
        #convert velocity from predict into coordinate
        particle_cord = predict_velocities.T * direction_vector + prev_position
        #print("particle_cord")
        #print(particle_cord)
        #get all blobs within threshold range
        euclidian_distances = np.sqrt(np.square(frame[['x']]-prev_position[0][0])['x'] + np.square(frame[['y']]-prev_position[0][1])['y'] )
        nearest_blobs = frame[euclidian_distances < threshold]
        
        #if there are no nearby blobs, exit update
        if nearest_blobs.shape == (0,1):
            #print("no blobs nearby!")
            return
        
        #if there is a single blob nearby
        elif nearest_blobs.shape[0] == 1:
            #print("one blob nearby")
            nearest_blobs = np.full(particle_cord.shape, nearest_blobs)
            #print(nearest_blobs)
            
        
        #if there are multiple blobs nearby, pick the best one by closest distance
        else:
            #print("more than one nearby blob")
            best_blob = np.zeros(particle_cord.shape)
            best_dist = np.full((particle_cord.shape[0],1), np.Infinity)
            for i, blob in nearest_blobs.iterrows():
                #print("blob #", i,": ", blob, blob.shape)
                dist = np.sqrt( np.square(np.full((particle_cord.shape[0]),blob[0])-particle_cord[:,0]) + np.square(np.full((particle_cord.shape[0]),blob[1])-particle_cord[:,1]))
                dist = np.array([dist]).T
                #print("dist: ", dist)
                best_blob[np.squeeze([dist < best_dist]),:] = blob
                #print("bestblob: ", best_blob)
                best_dist[dist < best_dist] = dist[dist < best_dist]
                #print("best_dist: ", best_dist)
            #print(nearest_blobs)
            nearest_blobs = best_blob
            
        nearest_blobs = pd.DataFrame(data=nearest_blobs, columns=["x", "y"])
            
        #update coordinates for each particle & blob
        
        #print("particle_cord")
        #print(particle_cord)
        #print(particle_cord.shape)
        #updated coordinate = (blobs + normalized random values * 0.8) + (predicted coordinates * 0.2)
        updated_cord = (nearest_blobs+np.random.standard_normal(nearest_blobs.shape)*noise)*alpha + particle_cord*(1-alpha)
        #print("Updated Cord")
        #print(updated_cord)
            
        #Calculate velocity for new coordinate
        #CHANGE THIS: change prev_position to blob_position
        new_dist = np.sqrt(np.square(updated_cord[['x']]-prev_position[0][0])['x'] + np.square(updated_cord[['y']]-prev_position[0][1])['y'] )
        new_velocities.append(new_dist/FPS)
        #print("new_dist: ", new_dist)
        #print(predict_weights.shape)
        #print(new_dist.shape)
        blob_dist = np.sqrt(np.square(updated_cord[['x']]-nearest_blobs[['x']])['x'] + np.square(updated_cord[['y']]-nearest_blobs[['y']])['y'] )
        #print("blob dist")
        #print(blob_dist)
        new_weights = (max(blob_dist)-blob_dist)/max(blob_dist)*alpha+(predict_weights.squeeze()*(1-alpha))
        new_weights = new_weights/sum(new_weights)

        #print("Ending velocities")
        #print(new_velocities)
        #print("Ending weights")
        #print(new_weights)
        self.velocities = np.array(new_velocities)
        self.weights = np.array(new_weights)
    
    def resample(self):
        
        # Define variables 
        velocities = self.velocities
        weights = self.weights
        des_num_particles = self.des_num_particles
        threshold = 1/des_num_particles
        
        # 1) Normalize the weights first (divide each weight by the sum of all the weights) 
        weights = np.divide(weights, np.sum(weights))
#       print("Normalized weights")
#       print(weights)

        # 2) Removing particles below threshold (< 1/N)
        if len(weights) <= des_num_particles: 
            print(weights)
            print(velocities)
            
            velocities = velocities.flatten()
            velocities = velocities[weights > 1/des_num_particles]
            boolean_mask = weights > 1/des_num_particles
            print("Boolean")
            print(boolean_mask)
            weights = weights[boolean_mask]
#         print("Removes particles below threshold (1/N)")
#         print(weights)

        # 3) Truncate (if greater than N). 
        # Ensure that we have our desired N particles, remove the lowest 
        if len(weights) > des_num_particles: 
            num = des_num_particles
            diff = len(weights) - des_num_particles

            for i in range(diff): 
                index = np.argmin(weights)
                np.delete(weights, index)
                np.delete(weights, index)
                
#         print("If greater than N, then we remove the lowest weight values")
#         print(weights) 
#         print(velocities)

        # 4) Normalize the weights that remain --> we have numbers that represent what portion of N that particular velocity has
        weights = np.divide(weights, np.sum(weights))
#         print("Normalizes the weights that remain")
#         print(weights)

        ### Shifting: if the number of particles is less than N...###
        # 5) Multiply the weights by the desired number of particles then take round
        # to get the number of particles we want representing those velocities and weights 

        # weights = weights*des_num_particles 
#         print("Proportion")
#         print(weights*des_num_particles)
#         print("Weights")
#         print(weights)
        proportion_particles = np.round_(weights*des_num_particles, decimals = 0, out=None)
        proportion_particles = list(proportion_particles.flatten())
#         print("Proportion of particles")
#         print(proportion_particles) #should output number of particles to replicate

        # 6) Duplicate the number of particles
        # TO DO: initialize new_weights?
        # at each index, replicate weight and particle at that many times

        updated_weights = np.repeat(weights, proportion_particles)
        updated_velocities = np.repeat(velocities, proportion_particles)
#         print("Duplicated particles")
#         print("Weights")
#         print(updated_weights)
#         print("Velocities")
#         print(updated_velocities)

        weights = updated_weights
        velocities = updated_velocities

        # 7) Set all the weights so that the weights are 1/total number of particles
        weights = np.full(weights.shape, 1/weights.size)
        #print("Final weights")
        #print(weights)
        assert np.sum(weights) == 1 # check that weights sum to 1          
        
        # Update velocities and weights
        self.weights = updated_weights
        self.velocities = updated_velocities
        
        #print("Updated velocity from resample function")
        #print(self.velocities)
        #print("Updated weights from resample function")
        #print(self.weights)
    
    def estimate(self):
        estimated_velocity = np.average(self.velocities, axis=1, weights=self.weights)
        self.estimated_velocity = estimated_velocity
        print("Print estimated velocity")
        print(self.estimated_velocity)

In [203]:
velocities = np.array([[0.0, 1.0, 2.0, 3.0, 4.0, 5.0]])
weights = np.array([[0.1, 0.3, 0.1, 0.1, 0.1, 0.4]])
prev_position = np.array([[2800,1600]])
prev_velocity = 5.0
direction_vector = np.array([[0.3, 0.5]])
estimated_location = np.array([[3400, 1400]])
threshold = 100
alpha = 0.8
noise = 8
des_num_particles = 8

In [204]:
particle_filter = Particle_Filter(estimated_location, velocities, weights, prev_velocity, direction_vector, des_num_particles)
# num_particles = desired number of particles
#   def __init__(self, estimated_location, init_velocities, weights, estimated_velocity, direction_vector):

all_frames = video['frame'].unique()
for num in all_frames:
    thisFrame = video.loc[video['frame'] == num]
    
    # for each frame, loop through each particle filter (one praticle filter per baboon)
    # then in for each baboon/particle filter (call predict, update, resample)
    
    particle_filter.update(thisFrame.iloc[:, 1:3], threshold, alpha, noise)
    particle_filter.resample()
    # particle_filter.estimate()

[0.12084535 0.28264999 0.07499065 0.08285532 0.12264248 0.31601621]
[[97.97315225 98.06991164 97.59065811 97.6583198  97.98664889 97.72366087]]
Boolean
[False  True False False False  True]


ValueError: operands could not be broadcast together with shapes (8,) (1,2) 

In [198]:
velocities = np.array([[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 0.8, 0.9]])
weights = np.array([[0.03, 0.4, 0.1, 0.5, 0.1, 0.2, 0.1, 0.5, 0.8, 0.9]]) 
print(velocities.size)
print(weights.size)

10
10


In [199]:
# TESTING BUBBLE

velocities = np.array([[0.0, 1.0, 2.0, 3.0, 4.0, 5.0]])
weights = np.array([[0.1, 0.3, 0.1, 0.1, 0.1, 0.4]])
des_num_particles = 5
threshold = 1/des_num_particles

print("This is the original weight")
print(weights)
        
# 1) Normalize the weights first (divide each weight by the sum of all the weights) 
weights = np.divide(weights, np.sum(weights))
print("Normalized weights")
print(weights)

# 2) Removing particles below threshold (< 1/N)
if len(weights) <= des_num_particles: 
    velocities = velocities[weights > 1/des_num_particles]
    boolean_mask = weights > 1/des_num_particles
    weights = weights[boolean_mask]
print("Removes particles below threshold (1/N)")
print(weights)

# 3) Truncate (if greater than N). 
# Ensure that we have our desired N particles, remove the lowest 
if len(weights) > des_num_particles: 
    num = des_num_particles
    diff = len(weights) - des_num_particles

    for i in range(diff): 
        index = np.argmin(weights)
        np.delete(weights, index)
        np.delete(weights, index)
print("If greater than N, then we remove the lowest weight values")
print(weights) 
print(velocities)

# 4) Normalize the weights that remain --> we have numbers that represent what portion of N that particular velocity has
weights = np.divide(weights, np.sum(weights))
print("Normalizes the weights that remain")
print(weights)

### Shifting: if the number of particles is less than N...###
# 5) Multiply the weights by the desired number of particles then take round
# to get the number of particles we want representing those velocities and weights 

# weights = weights*des_num_particles 
print("Proportion")
print(weights*des_num_particles)
print("Weights")
print(weights)
proportion_particles = np.round_(weights*des_num_particles, decimals = 0, out=None)
proportion_particles = list(proportion_particles.flatten())
print("Proportion of particles")
print(proportion_particles) #should output number of particles to replicate

# 6) Duplicate the number of particles
# TO DO: initialize new_weights?
# at each index, replicate weight and particle at that many times

updated_weights = np.repeat(weights, proportion_particles)
updated_velocities = np.repeat(velocities, proportion_particles)
print("Duplicated particles")
print("Weights")
print(updated_weights)
print("Velocities")
print(updated_velocities)

weights = updated_weights
velocities = updated_velocities

# 7) Set all the weights so that the weights are 1/total number of particles
weights = np.full(weights.shape, 1/weights.size)
print("Final weights")
print(weights)
assert np.sum(weights) == 1 # check that weights sum to 1        

#print("Updated velocity from resample function")
#print(self.velocities)
#print("Updated weights from resample function")
#print(self.weights)

This is the original weight
[[0.1 0.3 0.1 0.1 0.1 0.4]]
Normalized weights
[[0.09090909 0.27272727 0.09090909 0.09090909 0.09090909 0.36363636]]
Removes particles below threshold (1/N)
[0.27272727 0.36363636]
If greater than N, then we remove the lowest weight values
[0.27272727 0.36363636]
[1. 5.]
Normalizes the weights that remain
[0.42857143 0.57142857]
Proportion
[2.14285714 2.85714286]
Weights
[0.42857143 0.57142857]
Proportion of particles
[2.0, 3.0]
Duplicated particles
Weights
[0.42857143 0.42857143 0.57142857 0.57142857 0.57142857]
Velocities
[1. 1. 5. 5. 5.]
Final weights
[0.2 0.2 0.2 0.2 0.2]
